In [1]:
%cd ..

C:\Users\wuyua\Projects\PycharmProjects\RxExperiments


In [2]:
import numpy as np
import pandas as pd
import rx
from rx import operators
from rx import scheduler
from rx import subject

import logging
import datetime

from configs.config import GlobalConfig
GlobalConfig.initialize_global_configuration("configs/taurine.json")

from utils.logging import configure_logger_to_output
logging_output = configure_logger_to_output(level=logging.INFO)

# Description


This file contains the demonstration to use the notebook as the experiment terminal. It contains
- Image acquisition
- Temperature measurement
- FBRM reading
- RAMAN reading
- FP50 reading
- FP50 control
- Event logging widget

## Shared initialization

In [3]:
from utils.mqtt_wrapper import MQTTClientWrapper
from source.harvesters_source import HarvestersSource
from sink.save_image_sink import SaveImageSink
from sink.save_data_sink import SaveDataSink
from sink.visualization_sink import JupyterImageSink, PlotlyVisualizationSink
from datamodel.image import Image
from operators import data_framer
from plotly import graph_objects as go
from controls.camera_controller import CameraControl, CameraControlCommand
from controls.mqtt_fp50 import FP50Command, MQTTFP50Control
from controls.mqtt_pump import MQTTPump
from source.fbrm_source import FBRMSource
from source.raman_source import RamanSource
import ipywidgets as widgets


In [4]:
# MQTT connection
client = MQTTClientWrapper("experiment_taurine")
client.connect("192.168.43.1")
client.loop_start()

### Image analysis

In [5]:
# Image acquisition
image_acquisiton_enabled = False

# auto adjust will kick in if the average brightness is out of brightness_target +- deviation
brightness_target = 160
deviation = 5

image_source = HarvestersSource().pipe(operators.filter(lambda x: image_acquisiton_enabled), operators.share())
image_save_sink = SaveImageSink()
camera_control = CameraControl()
current_exposure = 2

# save image logic
image_save_subscription = image_source.subscribe(image_save_sink)

# camera controller logic
image_mean_brightness_source = image_source.pipe(
    operators.map(lambda x: x.image.mean())
)

def filter_brightness(x):
    hit_rate = np.bitwise_or(x > brightness_target + deviation, x < brightness_target - deviation).mean()
    mean_brightness = np.median(x)
    return hit_rate, mean_brightness

def prepare_camera_control_command(x):
    current_exposure = prepare_camera_control_command.current_exposure 
    current_exposure = current_exposure * brightness_target / x[1]
    if current_exposure > 50:
        current_exposure = 50
    if current_exposure < 0:
        current_exposure = 0
    prepare_camera_control_command.current_exposure = current_exposure
    logging.info(f"exposure auto control: brightness={x[1]}; exposure={current_exposure}")
    return CameraControlCommand().set_exposure(current_exposure)
prepare_camera_control_command.current_exposure = current_exposure # inject static variable

camera_auto_control_subscription = image_mean_brightness_source.pipe(
    operators.buffer_with_count(5), # buffer images
    operators.map(lambda x: np.asarray(x)), # convert into array.
    operators.map(filter_brightness),
    operators.filter(lambda x: x[0] >= 0.8), # only adjust if at least 4 out of 5 are not ok
    operators.map(prepare_camera_control_command)
).subscribe(camera_control)

# enable fan and trigger at the beginning
camera_control.on_command(CameraControlCommand().set_exposure(current_exposure).set_trigger(True))

# dataframed source
image_brightness_branch = image_mean_brightness_source.pipe(
    operators.sample(2),
    operators.map(lambda x: pd.DataFrame(data=[{"value": x}], index=[datetime.datetime.now()]))
)

# save file logic
mean_brightness_save_sink = SaveDataSink("image_brightness", "value")
mean_brightness_save_subscription = image_brightness_branch.subscribe(mean_brightness_save_sink)

In [6]:
# Front end panels
# camera visuallization logic
camera_image_sink = JupyterImageSink(name="Camera")
camera_visuallization_subscription = image_source.pipe(operators.sample(1)).subscribe(camera_image_sink)

# camera brightness visuallization logic
mean_brightness_sink = PlotlyVisualizationSink(name="Mean brightness")

mean_brightness_visuallization_subscription = image_brightness_branch.pipe(
    data_framer.data_framer(),
#     lttb.lttb_operator(600),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.value)])
).subscribe(mean_brightness_sink)

### Pump control

In [7]:
# back end
pump_control = MQTTPump(client, "pump", "6712580")
pump_control_dataframe = pump_control.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[{"data":x}]
        )
    )
)

pump_control_save = SaveDataSink("pump", "data")
pump_control_save_subscription = pump_control_dataframe.subscribe(pump_control_save)

In [8]:
# front end
pump_visualization_sink = PlotlyVisualizationSink(name="Pump Speed")

def comparer(x, y):
    np.all(x.values == y.values)
    
pump_visualization_subscription = pump_control_dataframe.pipe(
    data_framer.data_framer(),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.data, name="Slurry pump")])
).subscribe(pump_visualization_sink)
pump_visualization_sink.figure.update_layout(xaxis={"title": "Time"}, yaxis={"title": "Pump reading"})
;

''

## Water bath and temperature control

In [9]:
# Backend configuration

fp50_control = MQTTFP50Control(client)

fp50_dataframe_source = rx.combine_latest(fp50_control.pipe(operators.share())).pipe(
    operators.map(lambda x: pd.DataFrame(index=[datetime.datetime.now()], data=[{ 
        "temperature": x[0]["crystallizer_temperature"], 
        "setpoint": x[0]["setpoint"],
    }])),
)

# save file logic
fp50_save_sink = SaveDataSink("fp50", "data")
fp50_save_subscription = fp50_dataframe_source.subscribe(fp50_save_sink)

In [10]:
# front end panels
fp50_visualization_sink = PlotlyVisualizationSink(name="FP50")

fp50_visuallization_subscription = fp50_dataframe_source.pipe(
    data_framer.data_framer(300),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.values[:, 0], name=x.columns[0])])
).subscribe(fp50_visualization_sink)
fp50_visualization_sink.figure.update_layout(
    yaxis={"title": "$Temperature (^\circ C)$", "side": "left", "range": [10, 70]},
    xaxis={"title": "Time"}
)
;

''

### FBRM

In [11]:
# backend configuration
fbrm_source = FBRMSource(client, "fbrm").pipe(operators.share())

fbrm_dataframe_source = fbrm_source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[dict(zip(x["sizes"], x["counts"]))],
        )
    )
)

# save file logic
fbrm_save_sink = SaveDataSink("fbrm", "data")
fbrm_save_subscription = fbrm_dataframe_source.subscribe(fbrm_save_sink)

In [12]:
# front end configuration
fbrm_visuallization = PlotlyVisualizationSink(name="FBRM")
fbrm_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["sizes"], y=x["counts"], )])
).subscribe(fbrm_visuallization)

fbrm_visuallization.figure.update_layout(xaxis={"type":"log", "title":"$Size (\mu m)$"}, yaxis={"title": "Count"})

fbrm_count_visuallization = PlotlyVisualizationSink(name="FBRM statistics")
fbrm_count_visuallization_subscription = fbrm_source.pipe(
    operators.map(lambda x: pd.DataFrame(index=[datetime.datetime.now()], data=[{
        "total": np.sum(x["counts"])
    }])),
    data_framer.data_framer(),
    operators.map(lambda x: [go.Scatter(x=x.index, y=x.total, )])
).subscribe(fbrm_count_visuallization)
;

''

# Raman

In [13]:
# backend configuration
raman_source = RamanSource(client, "raman").pipe(operators.share())

raman_dataframe_source = raman_source.pipe(
    operators.map(
        lambda x: pd.DataFrame(
            index=[datetime.datetime.now()], 
            data=[dict(zip(x["wave_number"], x["data"]))],
        )
    )
)

# save file logic
raman_save_sink = SaveDataSink("raman", "data")
raman_save_subscription = raman_dataframe_source.subscribe(raman_save_sink)

In [14]:
# front end configuration
raman_visuallization = PlotlyVisualizationSink(name="Raman")
raman_visuallization_subscription = raman_source.pipe(
    operators.map(lambda x: [go.Scatter(x=x["wave_number"], y=x["data"], )])
).subscribe(raman_visuallization)

raman_visuallization.figure.update_layout(xaxis={"autorange": "reversed", "title":"$Wave number (cm^{-1})$"}, yaxis={"title": "Count"})

;

''

## Organize widgets

In [15]:
from ipywidgets import Layout, Box
# logging panel
# logging_output.layout = Layout(border="solid", width="100%", height="200px", overflow="scroll")

# image panel
image_panel_layout = Layout(display="flex", flex_flow="row", border="solid", width="100%")
mean_brightness_figure = Box(children=[mean_brightness_sink.fig], layout=Layout(flex="1 1 0%", width="auto"))
camera_image_sink.figure.layout = Layout(flex="1 1 0%", width="50%")
image_panel = Box(children=[mean_brightness_sink.figure, camera_image_sink.figure], layout=image_panel_layout)

# Pump control panel
pump_control_panel = Box(children=[pump_visualization_sink.figure], layout=Layout(display="flex", border="solid", width="100%"))

# temperature panel
temperature_panel = Box(children=[fp50_visualization_sink.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)

# FBRM panel
fbrm_panel = Box(children=[fbrm_visuallization.figure, fbrm_count_visuallization.figure], layout= 
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)

# Raman Panel
raman_panel = Box(children=[raman_visuallization.figure], layout=
    Layout(border="solid", width="100%", display="flex", flex_flow="column")
)


# control panel
event_logger_text = widgets.Text(
    description='Event annotation:',
    layout=Layout(width="100%"),
)
event_logger_text.on_submit(lambda x: logging.getLogger("annotation").info(x.value))


control_panel = Box(children=[event_logger_text], layout=Layout(border="solid", width="100%"))

# wrap up
panel = widgets.VBox([image_panel, pump_control_panel, temperature_panel, fbrm_panel, raman_panel, control_panel])

display(panel)

    'data': [], 'layout': {'template': '...', 'title': {'text': 'M…

## Stage 1: start up
Time 2020 Oct 4: 10:05PM
- RPM 300
- Heat up to 60degC to dissolve all

In [16]:
logging.getLogger("stage_reporter").info("stage 1 starts.")
threshold = 20

# fp50_control.on_command(FP50Command(60))
stage1_stop = subject.Subject()
stage1_output = widgets.Output()

@stage1_output.capture()
def check_fbrm_count(x: float):
    if x < threshold:
        print(f"Current FBRM count = {x}. Please initiate next step!")
        
fbrm_source.pipe(
    operators.map(lambda x: np.sum(x["counts"])),
    operators.take_until(stage1_stop),
).subscribe(check_fbrm_count, logging.error)

stage1_output

Output()

Stage 1 clean up code

In [17]:
logging.getLogger("stage_reporter").info("stage 1 ends.")

stage1_stop.on_next(True)
stage1_stop.on_completed()

## Stage 2: slow cooling to metastable zone

In [18]:
logging.getLogger("stage_reporter").info("stage 3 starts.")
pump_control.on_command(800)
image_acquisiton_enabled = True

# Conclusion notebook
1. seems like when sat @ 40, temp = 35, the wet milling can only get to 1000 fbrm reading.
2. what about use fbrm to do dnc, and use image analysis to evaluate the result?

In [19]:
brightness_target = 200

In [52]:
image_panel

Box(children=(FigureWidget({
    'data': [], 'layout': {'autosize': True, 'template': '...', 'title': {'text':…